# Imports

In [4]:
import math
import pandas as pd
import pennylane as qml
import time

from keras.datasets import mnist
from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [5]:
np.random.seed(131)
initial_params = np.random.random([45])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Import dataset

In [6]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
examples = np.append(train_X, test_X, axis=0)
examples = examples.reshape(70000, 28*28)
classes = np.append(train_y, test_y)

In [7]:
x = []
y = []
for (example, label) in zip(examples, classes):
    if label == 0:
        x.append(example)
        y.append(-1)
    elif label == 1:
        x.append(example)
        y.append(1)

In [8]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [9]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [10]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [11]:
device = qml.device("default.qubit", wires=8)

In [12]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    qml.U3(params[0], params[1], params[2], wires=0)
    qml.U3(params[3], params[4], params[5], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.U3(params[6], params[7], params[8], wires=2)
    qml.U3(params[9], params[10], params[11], wires=3)
    qml.CNOT(wires=[2, 3])
    qml.U3(params[12], params[13], params[14], wires=4)
    qml.U3(params[15], params[16], params[17], wires=5)
    qml.CNOT(wires=[5, 4])
    qml.U3(params[18], params[19], params[20], wires=6)
    qml.U3(params[21], params[22], params[23], wires=7)
    qml.CNOT(wires=[7, 6])

    # Second layer
    qml.U3(params[24], params[25], params[26], wires=1)
    qml.U3(params[27], params[28], params[29], wires=2)
    qml.CNOT(wires=[1, 2])
    qml.U3(params[30], params[31], params[32], wires=5)
    qml.U3(params[33], params[34], params[35], wires=6)
    qml.CNOT(wires=[6, 5])

    # Third layer
    qml.U3(params[36], params[37], params[38], wires=2)
    qml.U3(params[39], params[40], params[41], wires=5)
    qml.CNOT(wires=[2, 5])

    # Fourth layer
    qml.U3(params[42], params[43], params[44], wires=5)

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [13]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.65015361 0.94810917 0.38802889 0.64129616 0.69051205 0.12660931
 0.23946678 0.25415707 0.42644165 0.83900255 0.74503365 0.38067928
 0.26169292 0.05333379 0.43689638 0.20897912 0.59441102 0.09890353
 0.22409353 0.5842624  0.95908107 0.20988382 0.66133746 0.50261295
 0.32029143 0.12506485 0.80688893 0.98696002 0.54304141 0.23132314
 0.60351254 0.17669598 0.88653747 0.58902228 0.72117264 0.27567029
 0.78811469 0.1326223  0.39971595 0.62982409 0.42404345 0.16187284
 0.52034418 0.6070413  0.5808057 ]

Example features: [-3.20516464  1.54033847  1.92909414  0.58388829 -2.36304178  0.15431377
  0.87048378 -0.23291492]

Expectation value: -0.24596068929107134

 0: ──RY(-3.21)───Rot(0.388, 0.65, -0.388)─────Rϕ(0.388)───Rϕ(0.948)───╭C───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤     
 1: ──RY(1.54)────Rot(0.127, 0.641, -0.127)────Rϕ(0.127)───Rϕ(0.691)───╰X──Rot(0.8

# Accuracy test definition

In [14]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [15]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.43572241183162685
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.8065984072810011
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.8122866894197952
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.8111490329920364
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.7895335608646189
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.795221843003413
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.782707622298066
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.8156996587030717
best accuracy so far!
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 7

accuracy: 0.8054607508532423
test_accuracies: [0.782707622298066, 0.8156996587030717, 0.8145620022753128, 0.7940841865756542, 0.7986348122866894, 0.8088737201365188, 0.800910125142207, 0.8100113765642776, 0.8043230944254836, 0.8031854379977247, 0.8100113765642776, 0.8156996587030717, 0.7997724687144483, 0.7974971558589306, 0.795221843003413, 0.7997724687144483, 0.7929465301478953, 0.7838452787258248, 0.795221843003413, 0.782707622298066, 0.8054607508532423, 0.7963594994311718, 0.8225255972696246, 0.7963594994311718, 0.8031854379977247, 0.7986348122866894, 0.7974971558589306, 0.7918088737201365, 0.7974971558589306, 0.8054607508532423]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7906712172923778
test_accuracies: [0.8156996587030717, 0.8145620022753128, 0.7940841865756542, 0.7986348122866894, 0.8088737201365188, 0.800910125142207, 0.8100113765642776, 0.8043230944254836, 0.8031854379977247, 0.8100113765642776, 0.8156996587

epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7815699658703071
test_accuracies: [0.7997724687144483, 0.7974971558589306, 0.795221843003413, 0.7997724687144483, 0.7929465301478953, 0.7838452787258248, 0.795221843003413, 0.782707622298066, 0.8054607508532423, 0.7963594994311718, 0.8225255972696246, 0.7963594994311718, 0.8031854379977247, 0.7986348122866894, 0.7974971558589306, 0.7918088737201365, 0.7974971558589306, 0.8054607508532423, 0.7906712172923778, 0.8043230944254836, 0.7929465301478953, 0.7997724687144483, 0.800910125142207, 0.8077360637087599, 0.7963594994311718, 0.78839590443686, 0.7906712172923778, 0.7895335608646189, 0.7861205915813424, 0.7815699658703071]
epoch 471
epoch 472
epoch 473
epoch 474
epoch 475
epoch 476
epoch 477
epoch 478
epoch 479
epoch 480
accuracy: 0.795221843003413
test_accuracies: [0.7974971558589306, 0.795221843003413, 0.7997724687144483, 0.7929465301478953, 0.7838452787258248, 0.795221843003

In [16]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [ 7.76257357e-01  9.48109166e-01  4.51890400e-04  1.10616667e+00
  2.02740738e-02 -3.43246087e-03  3.93884011e-01  7.74999317e-02
 -6.51927192e-02  9.90084322e-01  1.72440060e-01 -4.11924425e-01
  4.53941273e-02 -1.46774883e+00  1.48071310e+00  3.59036254e-01
  5.27851160e-01 -1.01764908e+00  1.84383239e+00  3.41555952e-01
  1.47997959e+00  2.63426314e+00  6.61337460e-01  6.09480279e-02
 -5.44569971e-01  1.25064848e-01 -1.04408448e-03  8.74427015e-01
 -3.90776999e-02  5.46660082e-02  4.97656057e-01  1.51894908e-01
  8.19977613e-01  1.85684740e+00  7.21172645e-01  5.20027997e-01
  7.89511146e-01  1.32622301e-01 -6.09177404e-02  5.04436295e-01
  3.75286908e-01  5.18680344e-02  8.76680487e-01  6.07041298e-01
  9.02761098e-03]


# Testing

In [17]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7911592241768155


In [18]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 2.758111000061035
training time: 1340.566398382187
test time: 33.7648720741272
total time: 1377.0893814563751
